In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

# Translation

##  Data

In [2]:
import pandas as pd

In [3]:
combined_df_all = pd.read_csv('combined_df.csv', usecols=['EN', 'TI', 'EN_tokenized', 'TI_tokenized'])

In [4]:
combined_df_all.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
combined_df_all["EN"] = combined_df_all["EN"].str.replace(',', '')
combined_df_all["TI"] = combined_df_all["TI"].str.replace(',', '')
combined_df_all.reset_index(drop=True, inplace=True)
combined_df_all.to_csv('dat_1k.csv')

In [5]:
bi_dat_2 = pd.read_csv('bi_dat_2.csv', usecols = ['TI', 'EN', 'TI_tokenized', 'EN_tokenized'])
combined_df = pd.read_csv('combined_df.csv', usecols=['EN', 'TI', 'EN_tokenized', 'TI_tokenized'])

In [6]:
dat_5k = pd.concat([bi_dat_2, combined_df])

In [7]:

dat_5k.reset_index(drop=True, inplace=True)

In [8]:
dat_5k = dat_5k.iloc[:,[1,0,2,3]]

In [9]:
dat_5k.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
dat_5k["EN"] = dat_5k["EN"].str.replace(',', '')
dat_5k["TI"] = dat_5k["TI"].str.replace(',', '')

In [10]:
dat_5k.reset_index(drop=True, inplace=True)

In [11]:
dat_5k.to_csv('dat_5k.csv')

## Processing

In [13]:
import numpy as np
np.random.seed(1)
text_file = 'dat_5k.csv'
with open(text_file) as f:
    lines = f.read().split('\n')[:-1]

prefix = "translate English to Tigrinya: "
text_pairs = []
for line in lines:
    ind,orig, target = line.split(',')
    orig = orig.replace('"', '')
    target = target.replace('"', '')
    text_pairs.append({'orig': orig, 'target': target})

#Let's create some splits
np.random.shuffle(text_pairs)
num_valid_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_valid_samples
train_pairs = text_pairs[:num_train_samples]
valid_pairs = text_pairs[num_train_samples : num_train_samples + num_valid_samples]
test_pairs = text_pairs[num_train_samples + num_valid_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(valid_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

5036 total pairs
3526 training pairs
755 validation pairs
755 test pairs


In [14]:
train_pairs[0]

{'orig': 'Aaron shall be gathered to his people; for he shall not enter into the land which I have given to the children of Israel because you rebelled against my word at the waters of Meribah.',
 'target': 'ኣብ ማይ መሪባ ንትእዛዘይ ስለ ዝአቤኹምዎ፡ ኣሮን ናብቲ ንደቂ እስራኤል ዝሀብክዎም ምድሪ ኣይኣቱን እዩ እሞ፡ ናብ ሰቡ ይተአከብ።'}

In [15]:
train_list = []

In [16]:
for i in range(len(train_pairs)):
  train_list.append({'id': i, 'translation': {'en': train_pairs[i]['orig'],'ti':train_pairs[i]['target']}})

In [17]:
val_list = []
test_list = []

for i in range(len(valid_pairs)):
  val_list.append({'id': i, 'translation': {'en': valid_pairs[i]['orig'],'ti':valid_pairs[i]['target']}})

for i in range(len(test_pairs)):
  test_list.append({'id': i, 'translation': {'en': test_pairs[i]['orig'],'ti':test_pairs[i]['target']}})

## Preprocess

In [18]:
from transformers import AutoTokenizer


checkpoint = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [19]:
source_lang = "en"
target_lang = "ti"
prefix = "translate English to Tigrinya: "


def preprocess_function(examples):
    inputs = [prefix + example['translation'][source_lang] for example in examples]
    targets = [example['translation'][target_lang] for example in examples]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs
def preprocess_functionold(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [20]:
import tensorflow as tf
import pandas as pd
from datasets import Dataset

train_data = Dataset.from_list(train_list)
val_data = Dataset.from_list(val_list)
test_data = Dataset.from_list(test_list)

In [21]:
train_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 3526
})

In [22]:
tokenized_train = train_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/3526 [00:00<?, ? examples/s]

In [23]:
tokenized_val = val_data.map(preprocess_functionold, batched=True)
#tokenized_test = test_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/755 [00:00<?, ? examples/s]

In [24]:
#tokenized_test

In [25]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/mt5-base", return_tensors="tf")

## Evaluate

In [26]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00


In [27]:
import evaluate

metric = evaluate.load("bleu")

In [28]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Train

In [29]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [30]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

tf_model.h5:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [31]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_val_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [32]:
import tensorflow as tf
tf.random.set_seed(1234)
model.compile(optimizer=optimizer)  # No loss argument!

In [33]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_set)

In [35]:
callbacks = [metric_callback]

In [36]:
tokenizer.get_vocab() #['ዊ']

{'アドレス': 162173,
 '▁+1': 26988,
 '▁செய்த': 20962,
 '製作所': 164821,
 '肌が': 193513,
 '▁rohk': 40601,
 '▁rende': 61307,
 'atkal': 135294,
 'XUE': 189946,
 '9.3': 127740,
 'prokurat': 179301,
 'leone': 208992,
 '▁ספּ': 103953,
 '▁Меди': 134748,
 '剿': 240716,
 '仓': 100402,
 'liyin': 115596,
 '▁цивил': 161934,
 'enerale': 194422,
 '(82)': 130169,
 'ättä': 220014,
 'มีผล': 225439,
 'Signaler': 147427,
 'candidate': 53939,
 'だったのですが': 220473,
 'верт': 65887,
 'üren': 121910,
 '沏': 246901,
 '▁swar': 127560,
 '方は': 40306,
 'рург': 159895,
 'ⵖ': 241172,
 'ήσι': 122801,
 '▁อัตรา': 143950,
 '▁Special': 12183,
 '战士': 165843,
 '2880': 165851,
 'мога': 33781,
 'arrest': 165387,
 'wisseling': 198279,
 '=_.': 7266,
 '嚴': 182380,
 'ريق': 209975,
 '▁परिव': 123474,
 'familio': 160669,
 'cola': 33275,
 '\ue600': 240902,
 '8,"': 82851,
 '▁baha': 69322,
 'ყურ': 148098,
 'ांग': 178778,
 '▁Besk': 191975,
 '▁Tom': 4927,
 'omilla': 229637,
 'Flying': 98404,
 'viz': 24067,
 '▁6.9': 144476,
 'יאלי': 234694,
 '▁restr

In [37]:
model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=100) 

Epoch 1/100
220/220 [==============================] - 95s 230ms/step - loss: 11.4034 - val_loss: 5.5486
Epoch 2/100
220/220 [==============================] - 42s 193ms/step - loss: 6.3987 - val_loss: 5.0115
Epoch 3/100
220/220 [==============================] - 43s 194ms/step - loss: 5.6731 - val_loss: 4.6661
Epoch 4/100
220/220 [==============================] - 43s 194ms/step - loss: 5.3182 - val_loss: 4.3737
Epoch 5/100
220/220 [==============================] - 43s 193ms/step - loss: 5.0667 - val_loss: 4.1132
Epoch 6/100
220/220 [==============================] - 42s 193ms/step - loss: 4.8388 - val_loss: 3.8919
Epoch 7/100
220/220 [==============================] - 43s 194ms/step - loss: 4.6345 - val_loss: 3.7257
Epoch 8/100
220/220 [==============================] - 43s 194ms/step - loss: 4.4489 - val_loss: 3.5941
Epoch 9/100
220/220 [==============================] - 43s 193ms/step - loss: 4.2789 - val_loss: 3.4919
Epoch 10/100
220/220 [==============================] - 42s 193

## Inference

In [38]:
tokenized_test = test_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/755 [00:00<?, ? examples/s]

In [39]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [40]:
train_pairs[0:5]

[{'orig': 'Aaron shall be gathered to his people; for he shall not enter into the land which I have given to the children of Israel because you rebelled against my word at the waters of Meribah.',
  'target': 'ኣብ ማይ መሪባ ንትእዛዘይ ስለ ዝአቤኹምዎ፡ ኣሮን ናብቲ ንደቂ እስራኤል ዝሀብክዎም ምድሪ ኣይኣቱን እዩ እሞ፡ ናብ ሰቡ ይተአከብ።'},
 {'orig': 'Send you men that they may spy out the land of Canaan which I give to the children of Israel: of every tribe of their fathers shall you send a man everyone a prince among them.',
  'target': 'ነታ ኣነ ንደቂ እስራኤል ዝህቦም ምድሪ ኸነኣን ዚስልዩ ሰባት ስደድ፡ ከካብ ነገድ ኣቦታቶም ሓደ ሰብ፡ ኲሎም ካብቶም ሹማምቶም ይስደዱ።'},
 {'orig': 'In the greatness of your excellency you overthrow those who rise up against you. You send forth your wrath. It consumes them as stubble.',
  'target': 'ብዕቤት ግርማኻ ንዝተንስኡካ ጨፍለቕካዮም፡ ቍጥዓኻ ሰደድካ፡ ከም ሓሰር በልዓቶም።'},
 {'orig': 'You may make them an inheritance for your children after you to hold for a possession; of them may you take your slaves forever: but over your brothers the children of Israel you shal

In [42]:
from transformers import pipeline


In [43]:
from transformers import AutoTokenizer


In [44]:
from transformers import TFAutoModelForSeq2SeqLM


In [45]:
search_toks = {v:k for k, v in tokenizer.get_vocab().items()}


In [47]:
preds = []
trues = []
for text in test_pairs:
  inputs = tokenizer(text["orig"], return_tensors="tf").input_ids
  outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
  preds.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  trues.append(text["target"])

In [48]:
preds[0:10]

['ኪዳን እግዚኣብሄር ድማ ካብ ሓምሳን ሓምሳን ካብ ሰብን ካብ ሕሩድ ስጋ ወሰደ፡ ',
 'ካብ ማእከል ሓዊ እቲ ደበና ደው ኢልኩም መሊሱ፡ ነቲ ሰማይ ቓላት ዜብሉ ኣእማን ኣርኢኹ',
 'ኣምላኽ ከኣ ከምኡ ገበረ፡ በታ መዓልቲ ኸኣ ጠነሰት፡ ሳራይ ወዲውን ከም ሰበይቱ ሀቦ',
 'ካብ ቤት ኣቦይ ንመንፈስ ወዲ ኸነብር፡ ካብዚ ምድሪ እዚኣ ንዘርእኻ ኽህበ እየ',
 'ኵሎም ቊጹራት ሰፈር ኤፍሬም ንዅሎም ኻብ ሰፈር ኤፍሬም ዝቘጸርዎ፡ ሚእትን ሰላሳን ',
 'ሙሴ ኸኣ ነቲ ዕዮ ኣግማል ረኣየ፡ እንሆ ድማ፡ ከምቲ እግዚኣብሄር ዝኣዘዞ ዝገበሮ',
 'እዚ ኽትነብር ሕረድ እሞ፡ ድማ ንኣተኽና ኽትኸውን ኢና።',
 'ንመስዋእቲ ሓዊኻ፡ ከም ጽዋእ መቕደስ ዚበጽሖ፡ ከም ጭሔሎ መቕደስ ዓይኒ፡ ከም ',
 'እዚ ብርግጽ ምስ ተባዕታይ፡ ካብቲ ኻልኦት እዚኣቶም ዚገብር ካብቲ ዝገበሮ፡ ከምቲ ዝገበ',
 'ካብኡ ነቲ ሰብኣይ ድማ ናብ ሮቤል ኣውጽእዎ። ነቶም ሰባት ግና ከኣ ናብ ሳዶ']

In [49]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=preds, references=trues, max_order = 1)
print(results)

{'bleu': 0.18628075656607518, 'precisions': [0.2722352941176471], 'brevity_penalty': 0.6842637989678647, 'length_ratio': 0.7249466950959488, 'translation_length': 8500, 'reference_length': 11725}


In [50]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=preds, references=trues, max_order = 1)
print(results)

{'bleu': 0.18628075656607518, 'precisions': [0.2722352941176471], 'brevity_penalty': 0.6842637989678647, 'length_ratio': 0.7249466950959488, 'translation_length': 8500, 'reference_length': 11725}
